2023 - 2027

In [8]:
import pandas as pd
import os
import geopandas as gpd

In [4]:
new_path = '../../data/curated/2023_2027_data/'

if not os.path.exists(new_path):
    os.makedirs(new_path)

In [16]:
#sf2021 = gpd.read_file("../../data/raw/Geo/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")

# Slice the geolocation for Victoria
#COL_SF = ["SA2_CODE21", "geometry"]
#sf2021 = sf2021.loc[sf2021["STE_CODE21"] == '2']
#sf2021['geometry'] = sf2021['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
#sf2021 = sf2021[COL_SF]
#sf2021 = sf2021.dropna(subset=['geometry'])
#sf2021["SA2_CODE21"] = sf2021["SA2_CODE21"].astype(int)
#sf2021

In [15]:
#sf2016 = gpd.read_file("../../data/raw/Geo/1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp")

# Slice the geolocation for Victoria
#COL_SF = ["SA2_MAIN16", "geometry"]
#sf2016 = sf2016.loc[sf2016["STE_CODE16"] == '2']
#sf2016['geometry'] = sf2016['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
#sf2016 = sf2016[COL_SF]
#sf2016 = sf2016.dropna(subset=['geometry'])
#sf2016["SA2_MAIN16"] = sf2016["SA2_MAIN16"].astype(int)
#sf2016

,SA2_MAIN16,geometry
578,201011001,"POLYGON ((143.70477 -37.51935, 143.70482 -37.5..."
579,201011002,"POLYGON ((143.81896 -37.55583, 143.81644 -37.5..."
580,201011003,"POLYGON ((143.85013 -37.54247, 143.85011 -37.5..."
581,201011004,"POLYGON ((143.82821 -37.57559, 143.82840 -37.5..."
582,201011005,"POLYGON ((143.84170 -37.61597, 143.84175 -37.6..."
...,...,...
1035,217031476,"MULTIPOLYGON (((143.40263 -38.78153, 143.40252..."
1036,217041477,"POLYGON ((142.41438 -38.09304, 142.41399 -38.0..."
1037,217041478,"MULTIPOLYGON (((142.00869 -38.41716, 142.00876..."
1038,217041479,"POLYGON ((142.43668 -38.35545, 142.43658 -38.3..."


In [5]:
pred_gdp_sr = pd.read_csv('../../data/curated/feature_prediction/21_27_gdp_with_saving_predicted.csv')
pred_income = pd.read_csv('../../data/curated/feature_prediction/20_27_income_per_person_2016sa2.csv', index_col=[0])
pred_population = pd.read_csv('../../data/curated/feature_prediction/22_27_population.csv', index_col=[0])
pred_population = pred_population.rename(columns={'SA2 code': 'sa2_2021'})
pred_crime_cases = pd.read_csv('../../data/curated/feature_prediction/23_27_crime_case.csv', index_col=[0])
pred_crime_cases = pred_crime_cases.rename(columns={'Postcode': 'postcode'})

,Year,postcode,crime_count
0,2023,3000,15867
1,2023,3002,1089
2,2023,3003,1047
3,2023,3004,5324
4,2023,3006,3624
...,...,...,...
3460,2027,3506,112
3461,2027,3530,118
3462,2027,3647,81
3463,2027,3576,109


In [ ]:
year = [2023, 2024, 2025, 2026, 2027]

for i in year:
    data = pd.DataFrame()

    # store year, sa2 (2021 and 2016), postcode
    
    data['year'] = i

    # store gdp and saving rate into the dataframe
    gdp = list(pred_gdp_sr.loc[pred_gdp_sr['year'] == i, 'gdp'])[0]
    sr = list(pred_gdp_sr.loc[pred_gdp_sr['year'] == i, 'saving'])[0]
        
    data['gdp(USD Millioins)'] = gdp
    data['saving_rate(% of GDP)'] = sr

    # store income per person per year into the dataframe
    income_data = pred_income.loc[pred_income['Year'] == i][['sa2_2016', 'income_per_person_sa2']]
    data['income_per_person'] = (data.merge(income_data, on=['sa2_2016'], how='left')['income_per_person_sa2'])

    # store population density into the dataframe
    population_data = pred_population.loc[pred_population['year'] == i][['sa2_2021', 'pred']]
    data['population_density'] = (data.merge(population_data, on=['sa2_2021'], how='left')['pred'])

    # store crime cases into the dataframe
    crime_cases_data = pred_crime_cases.loc[pred_crime_cases['Year'] == i][['postcode', 'crime_count']]
    data['crime_cases'] = (data.merge(crime_cases_data, on=['postcode'], how='left')['crime_count'])


    data.to_csv(f'../../data/curated/2023_2027_data/{i}_data.csv')